In [1]:
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import glob,os,sys
from tqdm.auto import tqdm
import proplot as plot
import json,pickle
import dask.array as da
import gc
from sklearn.decomposition import PCA
sys.path.insert(1, '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/')
from tools import derive_var,read_and_proc,preproc_noensemble
from tools.mlr import mlr
from tools.preprocess import do_eof,preproc_maria,preproc_haiyan
%matplotlib inline
plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

/tmp/ipykernel_1665357/1321067960.py:20: ProplotWarning: rc setting 'linewidth' was renamed to 'meta.width' in version 0.8.
  plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',


#### Read

In [2]:
path = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/'
suffix = '_smooth_preproc_dict1b_g'

haiyan_u = np.concatenate([read_and_proc.depickle(path+'TCGphy/testML/output/haiyan/processed/uvwheat/'+'mem'+str(lime)+suffix)['u'] for lime in tqdm(range(1,21))],axis=0)
haiyan_v = np.concatenate([read_and_proc.depickle(path+'TCGphy/testML/output/haiyan/processed/uvwheat/'+'mem'+str(lime)+suffix)['v'] for lime in tqdm(range(1,21))],axis=0)
haiyan_w = np.concatenate([read_and_proc.depickle(path+'TCGphy/testML/output/haiyan/processed/uvwheat/'+'mem'+str(lime)+suffix)['w'] for lime in tqdm(range(1,21))],axis=0)
haiyan_qv = np.concatenate([read_and_proc.depickle(path+'TCGphy/testML/output/haiyan/processed/uvwheat/'+'mem'+str(lime)+suffix)['qv'] for lime in tqdm(range(1,21))],axis=0)
haiyan_theta = np.concatenate([read_and_proc.depickle(path+'TCGphy/testML/output/haiyan/processed/uvwheat/'+'mem'+str(lime)+suffix)['theta'] for lime in tqdm(range(1,21))],axis=0)
haiyan_heatsum = np.concatenate([read_and_proc.depickle(path+'TCGphy/testML/output/haiyan/processed/uvwheat/'+'mem'+str(lime)+suffix)['heatsum'] for lime in tqdm(range(1,21))],axis=0)
haiyan_hdia = np.concatenate([read_and_proc.depickle(path+'TCGphy/testML/output/haiyan/processed/uvwheat/'+'mem'+str(lime)+suffix)['hdia'] for lime in tqdm(range(1,21))],axis=0)
haiyan_rad = np.concatenate([read_and_proc.depickle(path+'TCGphy/testML/output/haiyan/processed/uvwheat/'+'mem'+str(lime)+suffix)['rad'] for lime in tqdm(range(1,21))],axis=0)
haiyan_ir = np.concatenate([read_and_proc.depickle(path+'TCGphy/testML/output/haiyan/processed/uvwheat/'+'mem'+str(lime)+suffix)['ir'] for lime in tqdm(range(1,21))],axis=0)

In [4]:
def build_a_xarray_dataset(ds=None,varname=None,varfile=None,dims=None,coords=None):
    def build_xa(arraydata=None,arrayname=None):
        da = xr.DataArray(data=arraydata,dims=dims,coords=coords,name=arrayname)
        return da
    # Complete the code
    for varobj,varnamen in zip(varfile,varname):
        da = build_xa(varobj,varnamen)
        ds = xr.merge( [ds , da ] )
    return ds

In [4]:
def build_a_xarray_dataset(ds=None,varname=None,varfile=None,dims=None,coords=None):
    def build_xa(arraydata=None,arrayname=None):
        da = xr.DataArray(data=arraydata,dims=dims,coords=coords,name=arrayname)
        return da
    # Complete the code
    for varobj,varnamen in zip(varfile,varname):
        da = build_xa(varobj,varnamen)
        ds = xr.merge( [ds , da ] )
    return ds

dims = ['sample','flatarray']
coords = dict(sample=np.linspace(0,haiyan_u.shape[0]-1,haiyan_u.shape[0]),flatarray=np.linspace(0,haiyan_u.shape[1]-1,haiyan_u.shape[1]))

ds = xr.Dataset(coords=coords)
haiyan_data=build_a_xarray_dataset(ds=ds,varname=['u','v','w','theta','heatsum','hdia','rad','qv','ir'],\
                                   varfile=[haiyan_u,haiyan_v,haiyan_w,haiyan_theta,haiyan_heatsum,haiyan_hdia,haiyan_rad,haiyan_qv,haiyan_ir],dims=dims,coords=coords)
del haiyan_u,haiyan_v,haiyan_w,haiyan_theta,haiyan_heatsum,haiyan_hdia,haiyan_rad
gc.collect()

99

#### Incremental PCA

In [5]:
from sklearn.decomposition import IncrementalPCA
def fit_cheap_pca(n_batches=None,n_comps=None,var=None):
    #n_batches = 20
    inc_pca = IncrementalPCA(n_components=n_comps)
    for X_batch in tqdm(np.array_split(var.data,n_batches)):
        inc_pca.partial_fit(X_batch)
    return inc_pca

In [6]:
pca_store = {}
for varnamex in (['rad','qv','ir']):#['u','v','w','theta','heatsum','hdia','rad','qv']):
    pca_store[varnamex] = fit_cheap_pca(n_batches=10,n_comps=150,var=haiyan_data[varnamex])

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [7]:
folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/haiyan/processed/'
read_and_proc.save_to_pickle(folderpath+'pca/PCA'+'_'+'dict2_g',pca_store,'PICKLE')

#### Fourier decomposition

In [5]:
utestfile = read_and_proc.depickle(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/'+'TCGphy/testML/output/haiyan/'+'urad/*')[0])
path_coor = '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/data/haiyan/U.nc'
coor = xr.open_dataset(path_coor)

In [6]:
utestfile[0].shape,haiyan_u.shape,coor.pres.shape

((95, 360, 208), (1565, 748800), (10,))

In [7]:
coorpres = coor.pres.data
mydata = xr.Dataset(data_vars=dict(u=(['samples','pressure','azimuth','radius'],haiyan_u.reshape(haiyan_u.shape[0],10,360,utestfile[0].shape[-1])),
                                   v=(['samples','pressure','azimuth','radius'],haiyan_v.reshape(haiyan_u.shape[0],10,360,utestfile[0].shape[-1])),
                                   w=(['samples','pressure','azimuth','radius'],haiyan_w.reshape(haiyan_u.shape[0],10,360,utestfile[0].shape[-1])),
                                   qv=(['samples','pressure','azimuth','radius'],haiyan_qv.reshape(haiyan_u.shape[0],10,360,utestfile[0].shape[-1])),
                                   theta=(['samples','pressure','azimuth','radius'],haiyan_theta.reshape(haiyan_u.shape[0],10,360,utestfile[0].shape[-1])),
                                   Q=(['samples','pressure','azimuth','radius'],haiyan_heatsum.reshape(haiyan_u.shape[0],10,360,utestfile[0].shape[-1])),
                                   RAD=(['samples','pressure','azimuth','radius'],haiyan_rad.reshape(haiyan_u.shape[0],10,360,utestfile[0].shape[-1])),
                                  IR=(['samples','pressure','azimuth','radius'],haiyan_ir.reshape(haiyan_u.shape[0],10,360,utestfile[0].shape[-1]))),
          coords=dict(pressure=coorpres),attrs=dict(description='Data for Fourier Analysis'))

In [31]:
import importlib
importlib.reload(preproc_haiyan)

<module 'tools.preprocess.preproc_haiyan' from '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/tools/preprocess/preproc_haiyan.py'>

In [33]:
_,kern,pres = preproc_haiyan.vertical_decomp(arraysize=10,coorpres=coorpres).get_coeff(mydata['Q'][90,:,90,20],True,None)
calc_model = {}
for indx,obj in tqdm(enumerate(['u','v','w','qv','theta','Q','RAD','IR'])):
    calc_model[obj] = preproc_haiyan.vertical_decomp(arraysize=10,coorpres=coorpres,kernIN=kern).processFFT(array=mydata[obj][...].data,vertlv_num=10)

0it [00:00, ?it/s]

In [53]:
folderpath='/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/testML/output/haiyan/processed/'
read_and_proc.save_to_pickle(folderpath+'spectral/vertdecomp'+'_'+'dict_g',calc_model,'PICKLE')